# 多层感知机（MLP）这一节需要掌握什么

---

## 一、核心一句话（必须记住）

> **多层感知机（MLP）= 在输入和输出之间加了"隐藏层" + "激活函数"，变成了非线性模型，可以解决 XOR 问题了。**

---

## 二、你必须掌握的内容（详细讲解）

### 🔴 掌握点 1：什么是隐藏层？

之前（线性回归、Softmax）：
```
输入 → 直接连接 → 输出
```

现在（多层感知机）：
```
输入 → 隐藏层 → 输出
       ↑
    中间多了一层"暗箱操作"
```

**用代码对比：**

```python
# 之前（Softmax分类）：只有 1 层
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 10)      # 输入 → 输出
)

# 现在（多层感知机）：有 2 层
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),    # 输入 → 隐藏层（256个神经元）
    nn.ReLU(),              # 激活函数 ← 新东西！
    nn.Linear(256, 10)      # 隐藏层 → 输出
)
```

**隐藏层的大小（256）是你自己定的：**

```
输入大小 = 784（图片决定的，不能改）
输出大小 = 10（类别数，不能改）
隐藏层大小 = 256 ← 你自己选！可以是 64、128、512...
```

---

### 🔴 掌握点 2：为什么必须要激活函数？

**如果不加激活函数会怎样？**

```python
# ❌ 没有激活函数的"多层"网络：
net = nn.Sequential(
    nn.Linear(784, 256),    # 第1层
    nn.Linear(256, 10)      # 第2层
)

# 数学上等价于：
# H = X × W1 + b1
# O = H × W2 + b2
#   = (X × W1 + b1) × W2 + b2
#   = X × (W1 × W2) + (b1 × W2 + b2)
#   = X × W' + b'
#        ↑    ↑
#     新权重  新偏置

# 结果：两层合并成了一层！
# 还是线性模型！
# 等于白加了一层！
```

**加了激活函数之后：**

```python
# ✅ 有激活函数的多层网络：
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),              # ← 激活函数打破了线性
    nn.Linear(256, 10)
)

# 数学上：
# H = ReLU(X × W1 + b1)   ← ReLU 是非线性的
# O = H × W2 + b2
#   = ReLU(...) × W2 + b2  ← 无法化简成 X × W' + b'

# 结果：真的是两层！能画曲线了！
```

**一句话记忆：**
> 激活函数 = 打破线性的"开关"，没有它，多层等于单层。

---

### 🔴 掌握点 3：三种常用激活函数

#### ① Sigmoid（现在不太用了）

```python
nn.Sigmoid()

# 公式：σ(x) = 1 / (1 + e^(-x))

# 效果：把任意数压缩到 (0, 1)
   输入 -100 → 输出 ≈ 0
   输入   0  → 输出 = 0.5
   输入 100  → 输出 ≈ 1

# 图形：
    1 ┤     ╭────
      │    ╱
  0.5 ┤   ╱
      │  ╱
    0 ┤─╯
      └───────────→
```

**缺点：计算慢（要算指数），现在很少用。**

---

#### ② Tanh（也不太用了）

```python
nn.Tanh()

# 公式：tanh(x) = (e^x - e^(-x)) / (e^x + e^(-x))

# 效果：把任意数压缩到 (-1, 1)
   输入 -100 → 输出 ≈ -1
   输入   0  → 输出 = 0
   输入 100  → 输出 ≈ 1

# 图形：
    1 ┤      ╭────
      │     ╱
    0 ┤────╯
      │   ╱
   -1 ┤──╯
      └───────────→
```

**缺点：也要算指数，慢。**

---

#### ③ ReLU（最常用！）

```python
nn.ReLU()

# 公式：ReLU(x) = max(0, x)
#     = x  如果 x > 0
#     = 0  如果 x ≤ 0

# 效果：小于0的全变成0，大于0的保持原值
   输入 -5  → 输出 0
   输入  0  → 输出 0
   输入  5  → 输出 5

# 图形：
      │    ╱
      │   ╱
      │  ╱
    0 ┤─────────
      └───────────→
```

**优点：**
- 超级快！只需要一个 `max` 运算，不用算指数
- 效果好
- 现在几乎都用它

**⚠️ 记住：如果不知道用什么，就用 ReLU！**

---

### 🔴 掌握点 4：多层感知机的完整结构

```python
# 单隐藏层（最基础的 MLP）：
net = nn.Sequential(
    nn.Flatten(),           # 拍平图片
    nn.Linear(784, 256),    # 输入层 → 隐藏层
    nn.ReLU(),              # 激活函数（打破线性）
    nn.Linear(256, 10)      # 隐藏层 → 输出层
)
```

**数据流向：**

```
图片 [1, 28, 28]
    │
    ▼ Flatten
[784]
    │
    ▼ Linear(784, 256)
[256]  ← 隐藏层的 256 个"神经元"
    │
    ▼ ReLU
[256]  ← 经过激活，变成非线性
    │
    ▼ Linear(256, 10)
[10]   ← 10 个类别的分数
```

---

### 🔴 掌握点 5：超参数怎么选？

**两个超参数：**
1. 隐藏层有几层？
2. 每层有多少个神经元？

#### 隐藏层数量

```python
# 单隐藏层（最简单）：
nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

# 多隐藏层（更复杂）：
nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),    # 第1个隐藏层
    nn.ReLU(),
    nn.Linear(256, 128),    # 第2个隐藏层
    nn.ReLU(),
    nn.Linear(128, 10)      # 输出层
)
```

**经验规则：**
- 简单任务 → 1 - 2 个隐藏层
- 复杂任务（比如图像识别）→ 更多层（但后面会学更好的模型 CNN）

#### 隐藏层大小

**李沐的建议（重点！）：**

```
输入 784 → 隐藏层 ? → 输出 10

方案1（单层，宽一点）：
  784 → [256] → 10

方案2（多层，逐渐变窄）：
  784 → [256] → [128] → [64] → 10
         ↓       ↓       ↓
       慢慢压缩数据维度

❌ 不要这样：
  784 → [64] → [256] → 10
         ↓
       先压太小，信息丢失了
```

**记忆口诀：**
> 宽度：逐渐递减（像漏斗一样，慢慢压缩信息）

**具体数字怎么选？**
- 没有固定规则，**靠试**
- 常见的：64, 128, 256, 512
- 从小开始试，不够再加大

---

### 🔴 掌握点 6：为什么能解决 XOR 问题？

**回顾：XOR 问题**

```
输入(0,0) → 0
输入(0,1) → 1
输入(1,0) → 1
输入(1,1) → 0

画在图上：
  y
  ↑
1 │ ○  ●   ← 同一类
  │
0 │ ●  ○   ← 另一类
  └────────→ x
    0  1
```

**单层（线性模型）：只能画直线，分不开**

**多层（MLP）：可以画曲线！**

李沐的解释：
```
步骤1：第一层学一条蓝线（根据 x 分）
       把 (0,?) 和 (1,?) 分开

步骤2：第二层学一条黄线（根据 y 分）
       把 (?,0) 和 (?,1) 分开

步骤3：第三层结合两条线的结果
       最终把 XOR 分对
```

**你只需要记住结论：**
> 多层 + 激活函数 = 能画曲线 = 能解决 XOR

---

## 三、你不需要掌握的内容（跳过）

| 内容 | 为什么不用管 |
|------|------------|
| Sigmoid/Tanh 的具体公式 | 现在不用它们了，用 ReLU |
| 为什么要有 e^x 这些数学 | 知道结论就行（会上溢下溢，所以慢） |
| XOR 的详细推导过程 | 知道"能解决"就够了 |
| 怎么证明加激活函数才能非线性 | 看一眼数学推导就行，不用背 |

---

## 四、对比表格（帮你理清关系）

| | 线性回归 | Softmax | 感知机 | 多层感知机 |
|---|---|---|---|---|
| 层数 | 1 层 | 1 层 | 1 层 | 2+ 层 |
| 激活函数 | ❌ 无 | ❌ 无 | sign(±1) | ✅ ReLU |
| 能画的边界 | 直线 | 直线 | 直线 | **曲线** |
| 能解决 XOR？ | ❌ | ❌ | ❌ | ✅ |
| 现在还用吗？ | ✅ | ✅ | ❌ | ✅ |

---

## 五、最精简的记忆模板

```python
# ========== 多层感知机（MLP）万能模板 ==========

net = nn.Sequential(
    nn.Flatten(),              # 🔒 图片必须先拍平
    
    # 🔴 隐藏层（重点！）
    nn.Linear(输入大小, 隐藏层大小),   # 第1层
    nn.ReLU(),                        # 🔴 激活函数，必须有！
    
    # 如果想要更多层，继续加：
    # nn.Linear(隐藏层大小, 更小的隐藏层),
    # nn.ReLU(),
    
    nn.Linear(隐藏层大小, 类别数)      # 🔒 输出层
)

# 🔴 分类：用 CrossEntropyLoss
loss = nn.CrossEntropyLoss()

# 🔴 优化器：和之前一样
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

# 🔴 训练循环：和之前一模一样
for epoch in range(10):
    for X, y in train_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
```

---

## 六、自查题（测试你是否真懂了）

### 题目1：填空

```python
# FashionMNIST 分类，单隐藏层 128 个神经元
net = nn.Sequential(
    nn.________(),
    nn.________(784, ___),
    nn.________(),           # 激活函数
    nn.________(128, ___)
)
```

<details>
<summary>点击查看答案</summary>

```python
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)
```
</details>

---

### 题目2：判断对错

```python
# 这个网络有几层？能画曲线吗？
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.Linear(256, 10)
)
```

<details>
<summary>点击查看答案</summary>

**错！虽然写了两个 Linear，但没有激活函数，数学上等价于 1 层。**

等价于：
```python
net = nn.Sequential(
    nn.Linear(784, 10)   # 只有这一层
)
```

**不能画曲线，还是线性模型。**

</details>

---

### 题目3：选激活函数

```
如果不知道用什么激活函数，应该选：
A. Sigmoid
B. Tanh
C. ReLU
D. 不用激活函数
```

<details>
<summary>点击查看答案</summary>

**C. ReLU**

理由：最快、效果好、大家都用。

</details>

---
